In [ ]:
import csv
import cv2
import numpy as np
from random import shuffle
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

samples = []
#with open('G:/Udacity/Project 3/data collection/driving_log.csv') as csvfile:
with open('C:/Users/HX/Self_drive_car_Project_3/data/data/driving_log_20180631_d.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
#samples = samples[1:16948]
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                #name = 'G:/Udacity/Project 3/data collection/IMG/'+batch_sample[0].split('\\')[-1]
                name = 'C:/Users/HX/Self_drive_car_Project_3/data/data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
            augmented_images, augmented_angles=[],[]
            for image, angle in zip(images, angles):
                augmented_images.append(image)
                augmented_angles.append(angle)
                augmented_images.append(cv2.flip(image,1))
                augmented_angles.append(angle*-1.0)
            # trim image to only see section with road
            X_train = np.array(augmented_images)
            y_train = np.array(augmented_angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)


"""
lines=[]
with open('G:/Udacity/Project 3/data collection/driving_log.csv') as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        lines.append(line)

#print(lines)
        
images=[]
measurements=[]
for line in lines:
    for i in range(3):
        source_path=line[i]
        #print(source_path)
        filename=source_path.split('\\')[-1]
        #print(filename)
        current_path='G:/Udacity/Project 3/data collection/IMG/'+filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3])
        measurements.append(measurement)

    
augmented_images, augmented_measurements=[],[]
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
    
x_train=np.array(augmented_images)
y_train=np.array(augmented_measurements)

"""

from keras.models import Sequential
from keras.layers import Flatten, Dense, InputLayer, Lambda, Dropout
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D
model=Sequential()
#model.add(InputLayer(input_shape=[160,320,3]))

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
#model.add(MaxPooling2D())
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
#model.add(MaxPooling2D())
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
#model.add(MaxPooling2D())
model.add(Convolution2D(64,3,3,activation="relu"))
#model.add(MaxPooling2D())
model.add(Convolution2D(64,3,3,activation="relu"))
#model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
#model.fit(x_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=3)



from keras.models import Model
import matplotlib.pyplot as plt

history_object = model.fit_generator(train_generator, samples_per_epoch =
    len(train_samples), validation_data = 
    validation_generator,
    nb_val_samples = len(validation_samples), 
    nb_epoch=4, verbose=1)
model.save('model.h5')
### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()


D:\Software\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
D:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:102: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
D:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
D:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:106: UserWarning: Update your `Conv2D` call to the Keras 2 AP

Epoch 1/4
16975/16975 [==============================] - 885s 52ms/step - loss: 0.0196 - val_loss: 0.0220
Epoch 2/4
16975/16975 [==============================] - 877s 52ms/step - loss: 0.0069 - val_loss: 0.0214
Epoch 3/4
16975/16975 [==============================] - 880s 52ms/step - loss: 0.0056 - val_loss: 0.0212
Epoch 4/4
16974/16975 [============================>.] - ETA: 0s - loss: 0.0050

In [ ]:
ls

In [ ]:
python drive.py model.h5

In [ ]:
model.h5
